## BigQuery Magic - Query results to Pandas Dataframe
The following code uses Jupyter **magic** to load the results of a BigQuery query into a Pandas dataframe.

Note that we do not need to import any libraries!

In [ ]:
%%bigquery stations_df --verbose
SELECT
  name, id,
  state,
  ST_GeogPoint(longitude, latitude) AS g
FROM
  `bigquery-public-data.ghcn_d.ghcnd_stations`
LIMIT
  1000

In [ ]:
stations_df.head()

## Location search via GIS
Now search for stations near Washinton DC.  You can set the lat long of another station in **ST_GeogPoint**

In [ ]:
%%bigquery dc_stations_df --verbose
WITH params AS (
  SELECT ST_GeogPoint(-77.0032, 38.9) AS center,
         50 AS maxn_stations,
         50 AS maxdist_km
),
distance_from_center AS (
  SELECT
    id,
    name,
    state,
    ST_GeogPoint(longitude, latitude) AS loc,
    ST_Distance(ST_GeogPoint(longitude, latitude), params.center) AS dist_meters
  FROM
    `bigquery-public-data.ghcn_d.ghcnd_stations`,
    params
  WHERE ST_DWithin(ST_GeogPoint(longitude, latitude), params.center, params.maxdist_km*1000)
)
SELECT * from distance_from_center

In [ ]:
dc_stations_df.head()

## Use parameters in BigQuery search
Set the **lat** and **long** as python variables.

Note that we need to cast the **FLOAT64** as **String**, since Jupyter **BigQuery Magic** mis-interprets the negative sign as a variable.

In [ ]:
long = -77.0032
lat  = 38.9
params = {"longitude": f'{long}', "latitude" : f'{lat}'}

Send the **JSON String** to the query via the **params** flag and then **CAST** them to **FLOAT64**.

In [ ]:
%%bigquery dc_stations_2_df --params $params --verbose
WITH params AS (
  SELECT ST_GeogPoint(CAST(@longitude as FLOAT64), CAST(@latitude as FLOAT64)) AS center,
         50 AS maxn_stations,
         50 AS maxdist_km
),
distance_from_center AS (
  SELECT
    id,
    name,
    state,
    longitude, 
    latitude,
    ST_Distance(ST_GeogPoint(longitude, latitude), params.center) AS dist_meters
  FROM
    `bigquery-public-data.ghcn_d.ghcnd_stations`,
    params
  WHERE ST_DWithin(ST_GeogPoint(longitude, latitude), params.center, params.maxdist_km*1000)
)
SELECT * from distance_from_center

In [ ]:
dc_stations_2_df.head()

In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd

In [ ]:
dc_geo_df = gpd.GeoDataFrame(dc_stations_2_df, geometry = gpd.points_from_xy(dc_stations_2_df.longitude, dc_stations_2_df.latitude))

In [ ]:
dc_geo_df.plot()

In [ ]:
!pip install descartes

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ax = world[world.continent == 'North America'].plot(
    color='white', edgecolor='black')
ax.set_xlim([-80, -75])
ax.set_ylim([36,40])
dc_geo_df.plot(ax=ax, color = 'red')